[![Ouvrir sur Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/SkatAI/deeplearning/blob/master/notebooks/deep_dream_claude.ipynb)

# DeepDream — Quand les réseaux de neurones hallucinent

**Deep Learning par la pratique — Alexis Perrier**

---

## Qu'est-ce que DeepDream ?

En 2015, des ingénieurs de Google se sont posé une question simple : **que "voit" un réseau de neurones quand il regarde une image ?**

Pour répondre, ils ont inversé le processus habituel :

| Processus normal | DeepDream |
|---|---|
| On modifie les **poids** du réseau pour qu'il reconnaisse mieux les images | On modifie **l'image elle-même** pour amplifier ce que le réseau y "voit" déjà |
| Descente de gradient sur les poids | **Montée** de gradient sur les pixels |
| Le réseau apprend | Le réseau **hallucine** |

Le résultat : des images psychédéliques où le réseau projette ses propres patterns — yeux, visages d'animaux, textures fractales — sur n'importe quelle photo.

### Pourquoi c'est intéressant ?

- C'est une fenêtre sur ce que les couches du réseau ont **réellement appris**
- Les couches basses voient des **textures et motifs géométriques**
- Les couches hautes voient des **formes complexes** (yeux, museaux, plumes)
- C'est aussi devenu un **outil artistique** à part entière

---

## Étape 0 — Setup

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.applications import inception_v3
import IPython.display as display
from PIL import Image
import time

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponible: {tf.config.list_physical_devices('GPU')}")

---

## Étape 1 — Charger une image de départ

On commence avec une image riche en textures et en détails — c'est ce qui donne les meilleurs résultats avec DeepDream.

In [ ]:
# Télécharger une image spectaculaire
url = 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg'
image_path = keras.utils.get_file('labrador.jpg', url)

# Fonctions utilitaires pour charger et afficher les images
def load_image(path, max_dim=512):
    """Charge une image et la redimensionne."""
    img = Image.open(path)
    img.thumbnail((max_dim, max_dim))
    img = np.array(img)
    return img

def show_image(img, title=''):
    """Affiche une image."""
    if isinstance(img, tf.Tensor):
        img = img.numpy()
    img = np.clip(img, 0, 255).astype(np.uint8)
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.axis('off')
    if title:
        plt.title(title, fontsize=16)
    plt.show()

# Charger et afficher
original_img = load_image(image_path)
show_image(original_img, 'Image originale')

---

## Étape 2 — Charger le modèle InceptionV3

DeepDream a été inventé avec le réseau **InceptionV3** de Google — c'est le modèle classique pour cet effet.

Ce réseau a été entraîné sur ImageNet (1,4 million d'images, 1000 catégories). Il a appris à reconnaître des textures, des formes, des objets, des animaux.

On ne va pas l'utiliser pour classifier des images. On va regarder **à l'intérieur** pour voir ce que chaque couche a appris.

In [ ]:
# Charger InceptionV3 pré-entraîné
base_model = inception_v3.InceptionV3(include_top=False, weights='imagenet')

# Lister quelques couches intéressantes
print(f"Nombre total de couches : {len(base_model.layers)}\n")
print("Quelques couches disponibles :")
for layer in base_model.layers:
    if 'mixed' in layer.name:
        print(f"  - {layer.name:20s}  →  output shape: {layer.output.shape}")

### 🤖 GEMINI

Demandez à Gemini :

> *"Qu'est-ce que le réseau InceptionV3 ? Pourquoi s'appelle-t-il Inception ? Quelle est la différence entre les couches 'mixed0' (début du réseau) et 'mixed10' (fin du réseau) en termes de ce qu'elles détectent ?"*

---

## Étape 3 — Comprendre le principe : gradient ascent

Quand on entraîne un réseau normalement, on fait de la **descente de gradient** : on modifie les poids pour **minimiser** une erreur.

Avec DeepDream, on fait l'inverse — de la **montée de gradient** (gradient ascent) :
- On choisit une couche du réseau
- On calcule les activations de cette couche pour notre image
- On modifie **les pixels de l'image** (pas les poids !) pour **maximiser** ces activations
- Le réseau amplifie ce qu'il "voit" déjà dans l'image

C'est comme demander au réseau : *"Qu'est-ce que tu vois ? Montre-le-moi encore plus fort !"*

### 3.1 — Choisir les couches

Le choix des couches détermine le type d'hallucinations :
- **Couches basses** (mixed0, mixed1) → textures, motifs géométriques, ondulations
- **Couches intermédiaires** (mixed3, mixed4) → formes répétitives, yeux, spirales
- **Couches hautes** (mixed7, mixed10) → objets complets, visages d'animaux, structures complexes

In [ ]:
# On choisit les couches dont on veut amplifier les activations
# Chaque couche a un poids qui contrôle son influence

dream_layers = {
    'mixed3': 0.5,
    'mixed5': 1.5,
}

# Créer le modèle DeepDream
outputs = [base_model.get_layer(name).output for name in dream_layers]
dream_model = keras.Model(inputs=base_model.input, outputs=outputs)

print("Couches sélectionnées pour le rêve :")
for name, weight in dream_layers.items():
    print(f"  - {name} (poids: {weight})")

### 3.2 — La fonction de perte et le gradient ascent

In [ ]:
def compute_loss(image, model, layer_weights):
    """Calcule la perte = somme pondérée des activations des couches choisies.
    Plus la perte est élevée, plus le réseau 'voit' des choses dans l'image."""
    # Pré-traitement pour InceptionV3
    img = inception_v3.preprocess_input(image)
    # Obtenir les activations des couches choisies
    activations = model(img)
    if not isinstance(activations, list):
        activations = [activations]

    losses = []
    weights = list(layer_weights.values())
    for activation, weight in zip(activations, weights):
        # La perte = la moyenne des activations × le poids
        loss = tf.reduce_mean(activation) * weight
        losses.append(loss)

    return tf.reduce_sum(losses)


@tf.function
def deepdream_step(image, model, layer_weights, step_size):
    """Une étape de gradient ascent : modifier l'image pour amplifier les activations."""
    with tf.GradientTape() as tape:
        tape.watch(image)
        loss = compute_loss(image, model, layer_weights)

    # Calculer le gradient par rapport aux pixels de l'image
    gradients = tape.gradient(loss, image)

    # Normaliser les gradients
    gradients /= tf.math.reduce_std(gradients) + 1e-8

    # MONTÉE de gradient : on AJOUTE le gradient (au lieu de le soustraire)
    image = image + gradients * step_size

    # Garder les valeurs de pixels dans une plage raisonnable
    image = tf.clip_by_value(image, -1, 1)

    return image, loss

print("Fonctions DeepDream prêtes !")

### ❓ QUESTION

Regardez la ligne `image = image + gradients * step_size` dans le code ci-dessus.

- Dans l'entraînement classique d'un réseau, on écrirait `weights = weights **-** learning_rate * gradients`. Quelle est la différence ?
- Pourquoi ajoute-t-on le gradient au lieu de le soustraire ?

*(votre réponse ici)*

---

## Étape 4 — Premier rêve !

On lance DeepDream sur notre image. On va voir le réseau projeter ses hallucinations sur la photo.

In [ ]:
def run_deepdream_simple(image, model, layer_weights, steps=100, step_size=0.01):
    """Lance DeepDream sur une image pendant un certain nombre d'étapes."""
    # Convertir en float32 et ajouter la dimension batch
    img = tf.constant(np.array(image), dtype=tf.float32)
    img = tf.expand_dims(img, axis=0)

    for step in range(steps):
        img, loss = deepdream_step(img, model, layer_weights, step_size)
        if step % 25 == 0:
            print(f"  Étape {step:3d}, perte = {loss.numpy():.2f}")

    # Retirer la dimension batch et remettre en [0, 255]
    result = img[0].numpy()
    result = ((result + 1) / 2 * 255)  # de [-1,1] vers [0,255]
    result = np.clip(result, 0, 255).astype(np.uint8)
    return result


print("🌀 DeepDream en cours...")
start_time = time.time()

dream_img = run_deepdream_simple(
    original_img,
    dream_model,
    dream_layers,
    steps=100,
    step_size=0.01
)

elapsed = time.time() - start_time
print(f"\n✅ Terminé en {elapsed:.1f} secondes")

# Afficher côte à côte
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
ax1.imshow(original_img)
ax1.set_title('Image originale', fontsize=14)
ax1.axis('off')
ax2.imshow(dream_img)
ax2.set_title('DeepDream', fontsize=14)
ax2.axis('off')
plt.tight_layout()
plt.show()

### ❓ QUESTION

- Quels patterns voyez-vous apparaître dans l'image ?
- Reconnaissez-vous des formes d'animaux, des yeux, des textures ?
- Pourquoi le réseau projette-t-il ces formes en particulier ? (indice : sur quel dataset a-t-il été entraîné ?)

*(vos observations ici)*

---

## Étape 5 — Explorer les couches : que voit le réseau à chaque profondeur ?

C'est la partie la plus révélatrice. On va appliquer DeepDream avec **différentes couches** pour voir comment les features évoluent dans le réseau.

In [ ]:
# Tester 4 configurations de couches différentes
experiments = {
    'Couches basses\n(textures)': {'mixed0': 1.0, 'mixed1': 1.0},
    'Couches moyennes\n(motifs)': {'mixed3': 1.0, 'mixed4': 1.0},
    'Couches hautes\n(formes)': {'mixed5': 1.0, 'mixed7': 1.0},
    'Couches très hautes\n(objets)': {'mixed8': 1.0, 'mixed10': 1.0},
}

fig, axes = plt.subplots(1, 4, figsize=(24, 6))

for ax, (title, layers_config) in zip(axes, experiments.items()):
    print(f"\n🌀 {title.replace(chr(10), ' ')}...")

    # Créer le modèle pour ces couches
    layer_outputs = [base_model.get_layer(name).output for name in layers_config]
    model = keras.Model(inputs=base_model.input, outputs=layer_outputs)

    # Lancer DeepDream
    result = run_deepdream_simple(
        original_img, model, layers_config,
        steps=80, step_size=0.01
    )

    ax.imshow(result)
    ax.set_title(title, fontsize=13)
    ax.axis('off')

plt.suptitle('DeepDream — Impact de la profondeur des couches', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

### ❓ QUESTION

Comparez les 4 images :
- **Couches basses** : quel type de patterns voyez-vous ? (textures ? motifs géométriques ?)
- **Couches hautes** : les patterns sont-ils plus figuratifs ? Reconnaissez-vous des formes ?
- Que nous apprend cette progression sur la façon dont un réseau de neurones "comprend" une image ?

*(vos observations ici)*

---

## Étape 6 — DeepDream multi-scale (octaves)

La version basique produit des résultats un peu flous. La technique des **octaves** donne des résultats bien plus nets et spectaculaires.

Le principe :
1. Réduire l'image
2. Appliquer DeepDream sur la petite version
3. Agrandir le résultat
4. Ajouter les détails perdus
5. Réappliquer DeepDream
6. Répéter à plusieurs échelles

Chaque passage à une échelle supérieure s'appelle une **octave** (comme en musique).

In [ ]:
def run_deepdream_octaves(image, model, layer_weights,
                          steps_per_octave=50, step_size=0.01,
                          num_octaves=3, octave_scale=1.3):
    """DeepDream multi-scale avec octaves pour des résultats plus nets."""

    img = tf.constant(np.array(image), dtype=tf.float32)
    img = tf.expand_dims(img, axis=0)

    original_shape = tf.shape(img)[1:3]

    for octave in range(num_octaves):
        # Calculer la taille pour cette octave
        scale = octave_scale ** (num_octaves - 1 - octave)
        new_size = tf.cast(tf.cast(original_shape, tf.float32) / scale, tf.int32)

        # Redimensionner l'image
        img = tf.image.resize(img, new_size)

        print(f"  Octave {octave + 1}/{num_octaves} — taille: {new_size[0].numpy()}x{new_size[1].numpy()}")

        # Appliquer DeepDream à cette échelle
        for step in range(steps_per_octave):
            img, loss = deepdream_step(img, model, layer_weights, step_size)

        print(f"    → perte = {loss.numpy():.2f}")

    # Remettre à la taille originale
    img = tf.image.resize(img, original_shape)

    # Post-traitement
    result = img[0].numpy()
    result = ((result + 1) / 2 * 255)
    result = np.clip(result, 0, 255).astype(np.uint8)
    return result


# Recréer le modèle avec les couches de rêve
dream_layers_octave = {'mixed3': 0.5, 'mixed5': 1.5}
outputs_octave = [base_model.get_layer(name).output for name in dream_layers_octave]
dream_model_octave = keras.Model(inputs=base_model.input, outputs=outputs_octave)

print("🌀 DeepDream multi-scale (octaves) en cours...")
start_time = time.time()

dream_octave = run_deepdream_octaves(
    original_img,
    dream_model_octave,
    dream_layers_octave,
    steps_per_octave=50,
    step_size=0.01,
    num_octaves=3,
    octave_scale=1.3
)

elapsed = time.time() - start_time
print(f"\n✅ Terminé en {elapsed:.1f} secondes")

# Comparer les 3 versions
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 7))
ax1.imshow(original_img)
ax1.set_title('Originale', fontsize=14)
ax1.axis('off')
ax2.imshow(dream_img)
ax2.set_title('DeepDream simple', fontsize=14)
ax2.axis('off')
ax3.imshow(dream_octave)
ax3.set_title('DeepDream octaves', fontsize=14)
ax3.axis('off')
plt.tight_layout()
plt.show()

### ❓ QUESTION

Comparez les versions "simple" et "octaves" :
- Laquelle produit des détails plus fins ?
- La version octaves contient-elle des patterns à différentes échelles ?

*(vos observations ici)*

---

## Étape 7 — Expérimenter

### 7.1 — Modifier l'intensité du rêve

Le paramètre `step_size` contrôle l'intensité de l'effet. Plus il est élevé, plus les hallucinations sont prononcées.

In [ ]:
intensities = [0.005, 0.01, 0.03]
labels = ['Subtil', 'Modéré', 'Intense']

fig, axes = plt.subplots(1, 3, figsize=(20, 7))

for ax, step_size, label in zip(axes, intensities, labels):
    print(f"\n🌀 {label} (step_size={step_size})...")
    result = run_deepdream_simple(
        original_img, dream_model, dream_layers,
        steps=80, step_size=step_size
    )
    ax.imshow(result)
    ax.set_title(f'{label}\n(step_size={step_size})', fontsize=13)
    ax.axis('off')

plt.suptitle("Impact de l'intensité (step_size)", fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

### 🤖 GEMINI — Expérimenter avec d'autres combinaisons

Demandez à Gemini :

> *"Modifie le dictionnaire `dream_layers` pour utiliser uniquement la couche 'mixed7' avec un poids de 2.0. Relance DeepDream avec 100 étapes et step_size=0.01. Affiche le résultat à côté de l'original."*

Puis essayez d'autres combinaisons et demandez :

> *"Quelles couches d'InceptionV3 produisent les effets les plus psychédéliques ? Pourquoi les couches profondes génèrent-elles des formes d'animaux ?"*

In [ ]:
# VOTRE CODE GÉNÉRÉ PAR GEMINI ICI



---

## Étape 8 — Testez avec vos propres images !

Uploadez n'importe quelle image et voyez ce que le réseau y hallucine.

**Suggestions** :
- une de vos créations 3D / Blender
- un paysage, un bâtiment, un portrait
- une texture, un motif abstrait
- une photo du quotidien

Les images avec beaucoup de textures et de détails donnent les résultats les plus spectaculaires.

In [ ]:
from google.colab import files

print("Uploadez une image :")
uploaded = files.upload()

In [ ]:
for filename in uploaded.keys():
    print(f"\n📷 Image : {filename}")

    # Charger l'image
    my_img = load_image(filename, max_dim=512)

    # Lancer DeepDream avec octaves
    my_layers = {'mixed3': 0.5, 'mixed5': 1.5}
    my_outputs = [base_model.get_layer(name).output for name in my_layers]
    my_model = keras.Model(inputs=base_model.input, outputs=my_outputs)

    print("🌀 DeepDream en cours...")
    my_dream = run_deepdream_octaves(
        my_img, my_model, my_layers,
        steps_per_octave=50, step_size=0.01,
        num_octaves=3, octave_scale=1.3
    )

    # Afficher
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
    ax1.imshow(my_img)
    ax1.set_title('Votre image', fontsize=14)
    ax1.axis('off')
    ax2.imshow(my_dream)
    ax2.set_title('DeepDream', fontsize=14)
    ax2.axis('off')
    plt.tight_layout()
    plt.show()

### 🤖 GEMINI — Aller plus loin avec vos images

Demandez à Gemini :

> *"Modifie le code pour tester mon image avec 3 configurations de couches différentes côte à côte : couches basses (mixed0, mixed1), couches moyennes (mixed3, mixed5), couches hautes (mixed8, mixed10). Affiche les 3 résultats dans une grille."*

Ou bien :

> *"Ajoute le code pour sauvegarder l'image DeepDream en haute résolution dans un fichier PNG."*

In [ ]:
# VOTRE CODE GÉNÉRÉ PAR GEMINI ICI



---

## Étape 9 — Synthèse

### Ce qu'on a appris

| Concept | Ce qu'on a vu |
|---|---|
| **Gradient ascent** | Au lieu de minimiser une erreur, on maximise les activations → le réseau amplifie ce qu'il "voit" |
| **Hiérarchie des features** | Les couches basses détectent des textures, les couches hautes des objets complexes |
| **Biais du dataset** | Le réseau hallucine des animaux parce qu'il a été entraîné sur ImageNet (beaucoup d'animaux) |
| **Octaves** | Travailler à plusieurs échelles produit des détails plus fins et plus nets |

### ❓ QUESTIONS FINALES

1. Si on entraînait le réseau sur un dataset de bâtiments au lieu d'ImageNet, que verrait-on apparaître dans les rêves ?

2. DeepDream est souvent présenté comme un outil artistique. Mais en quoi est-ce aussi un outil de **compréhension** des réseaux de neurones ?

3. Comment pourriez-vous intégrer DeepDream dans un projet créatif (3D, design, art génératif) ?

*(vos réponses ici)*

---

## Pour aller plus loin (optionnel)

Quelques pistes d'exploration avec Gemini :

- **Créer une animation** : appliquer DeepDream de manière itérative (chaque output devient l'input suivant) et assembler les frames en GIF
- **Style transfer** : combiner DeepDream avec le Neural Style Transfer pour appliquer le style d'un tableau sur une photo
- **Rêve progressif** : augmenter graduellement le step_size pour montrer l'image qui "plonge" dans le rêve
- **Zoomer dans le rêve** : appliquer DeepDream + zoom au centre à chaque itération (effet de plongée infinie)

In [ ]:
# VOTRE EXPLORATION ICI

